In [1]:
rawUserData=sc.textFile("hdfs://127.0.0.1:9000/ml-100k/u.data")
rawUserData.count()

100000

In [2]:
rawUserData.first()
#User id, Project id, Rate, Data

'196\t242\t3\t881250949'

In [6]:
from pyspark.mllib.recommendation import Rating

#get [:3] : User id, Project id, Rate
rawRatings=rawUserData.map(lambda line:line.split("\t")[:3])
rawRatings.take(5)

[['196', '242', '3'],
 ['186', '302', '3'],
 ['22', '377', '1'],
 ['244', '51', '2'],
 ['166', '346', '1']]

In [10]:
#prepare ALS training data
ratingsRDD=rawRatings.map(lambda x:(x[0],x[1],x[2]))
ratingsRDD.take(5)


[('196', '242', '3'),
 ('186', '302', '3'),
 ('22', '377', '1'),
 ('244', '51', '2'),
 ('166', '346', '1')]

In [11]:
numRatings=ratingsRDD.count()
numRatings

100000

In [12]:
numUsers=ratingsRDD.map(lambda x:x[0]).distinct().count()
numUsers

943

In [14]:
numMovies=ratingsRDD.map(lambda x:x[1]).distinct().count()
numMovies

1682

In [16]:
#traning model
from pyspark.mllib.recommendation import ALS
model=ALS.train(ratingsRDD,10,10,0.01)
print(model)

In [17]:
model.recommendProducts(100,5)

[Rating(user=100, product=1512, rating=6.290521379386166),
 Rating(user=100, product=865, rating=6.191215745388112),
 Rating(user=100, product=1503, rating=6.156079244343146),
 Rating(user=100, product=1094, rating=6.073465595675719),
 Rating(user=100, product=1157, rating=6.065513082601354)]

In [18]:
model.predict(100,1141)

4.319059020931728

In [19]:
model.recommendUsers(product=100,num=5)

[Rating(user=362, product=100, rating=6.701001121214192),
 Rating(user=34, product=100, rating=6.358904269946141),
 Rating(user=448, product=100, rating=6.006002080879295),
 Rating(user=820, product=100, rating=5.894339758881064),
 Rating(user=772, product=100, rating=5.846718246599149)]

In [20]:
itemsRDD=sc.textFile("hdfs://127.0.0.1:9000/ml-100k/u.item")
itemsRDD.count()

1682

In [21]:
movieTitle=itemsRDD.map(lambda line:line.split("|")).map(lambda a:(float(a[0]),a[1])).collectAsMap()
len(movieTitle)

1682

In [23]:
recommendP=model.recommendProducts(100,5)
for p in recommendP:
    print("对用户:"+str(p[0])+",推荐电影: "+str(movieTitle[p[1]])+",推荐评分:"+str(p[2]))

对用户:100,推荐电影: World of Apu, The (Apur Sansar) (1959),推荐评分:6.290521379386166
对用户:100,推荐电影: Ice Storm, The (1997),推荐评分:6.191215745388112
对用户:100,推荐电影: Gold Diggers: The Secret of Bear Mountain (1995),推荐评分:6.156079244343146
对用户:100,推荐电影: Thin Line Between Love and Hate, A (1996),推荐评分:6.073465595675719
对用户:100,推荐电影: Relic, The (1997),推荐评分:6.065513082601354


In [24]:
recommendP=model.recommendProducts(10,5)
for p in recommendP:
    print("对用户:"+str(p[0])+",推荐电影: "+str(movieTitle[p[1]])+",推荐评分:"+str(p[2]))

对用户:10,推荐电影: Pather Panchali (1955),推荐评分:5.7628261365182345
对用户:10,推荐电影: Angel Baby (1995),推荐评分:5.667407892712781
对用户:10,推荐电影: Boys, Les (1997),推荐评分:5.460790042228479
对用户:10,推荐电影: Fresh (1994),推荐评分:5.30026421679077
对用户:10,推荐电影: So Dear to My Heart (1949),推荐评分:5.288921446286993
